# Quantum Agent Manager

Quantum Agent Manager is a quantum-inspired task scheduling system for managing a swarm of autonomous agents. This notebook demonstrates an end-to-end pipeline—from formulating a scheduling problem as a Quadratic Unconstrained Binary Optimization (QUBO) model to submitting it via the Azure Quantum CLI, retrieving and interpreting the results, and integrating CrewAI using a ReAct-based approach to refine the quantum-optimized schedule into a final actionable plan. The system uses real Azure Quantum services and CrewAI for dynamic, autonomous task management.

In [ ]:
!pip install azure-quantum ipywidgets crewai

## Step 1: Environment Setup

Log in to Azure and set your Quantum workspace using the following commands. In this example, we use the following actual values:
- **Subscription ID:** 12345678-1234-1234-1234-123456789012
- **Resource Group:** QuantumRG
- **Workspace Name:** QuantumWorkspace
- **Location:** eastus

In [ ]:
!az login
!az account set --subscription "12345678-1234-1234-1234-123456789012"
!az quantum workspace set -g "QuantumRG" -w "QuantumWorkspace" -l "eastus"

## Step 1.5: CrewAI Integration using ReAct

We now integrate CrewAI using a ReAct-based approach. The CrewAI agent will use the GPT-3.5-turbo model to reason about and refine the schedule produced by the quantum optimization process.

In [ ]:
import crewai

# Initialize the CrewAI agent with ReAct using the real model
agent = crewai.Agent(name="QuantumSchedulerAgent", model="gpt-3.5-turbo", chain_type="ReAct")

# This function sends a prompt to the CrewAI agent to refine the schedule
def refine_schedule(schedule):
    prompt = f"Refine the following task schedule for optimal load balancing and minimal makespan: {schedule}"
    refined = agent.act(prompt)
    return refined

# Later, after obtaining the quantum solution, we will use refine_schedule() to improve it.

## Step 2: QUBO Formulation for Task Scheduling

We model the scheduling problem as a QUBO. In this example, we have:
- **Tasks (N):** 2
- **Agents (M):** 2
- **Time Slots (T):** 2

Each binary variable indicates whether a task is assigned to a specific agent at a specific time slot. Penalty terms ensure that each task is assigned exactly once and that no agent has overlapping tasks, while a reward term encourages valid assignments.

In [ ]:
import json

# Define problem parameters
N = 2  # number of tasks
M = 2  # number of agents
T = 2  # number of time slots

# Helper function: map (task, agent, time) to a unique index
def var_index(task, agent, time):
    return task * (M * T) + agent * T + time

# List to hold QUBO terms
terms = []

# Set penalty and reward weights
A = 10.0  # penalty for multiple assignments of a task
B = 10.0  # penalty for agent overlap
reward = -1.0  # reward for a valid assignment

# Constraint: Each task must be assigned exactly once
for task in range(N):
    indices = [var_index(task, agent, time) for agent in range(M) for time in range(T)]
    for i in range(len(indices)):
        for j in range(i+1, len(indices)):
            terms.append({"indices": [indices[i], indices[j]], "c": A})

# Constraint: No agent gets two tasks at the same time
for agent in range(M):
    for time in range(T):
        indices = [var_index(task, agent, time) for task in range(N)]
        for i in range(len(indices)):
            for j in range(i+1, len(indices)):
                terms.append({"indices": [indices[i], indices[j]], "c": B})

# Objective: Reward each assignment to encourage scheduling
for task in range(N):
    for agent in range(M):
        for time in range(T):
            terms.append({"indices": [var_index(task, agent, time)], "c": reward})

# Build the QUBO problem JSON
qubo_problem = {
    "problemType": "pubo",
    "name": "TaskSchedulingQUBO",
    "terms": terms
}

# Save the QUBO problem to 'problem.json'
with open('problem.json', 'w') as f:
    json.dump(qubo_problem, f, indent=2)

print('QUBO problem saved to problem.json')

## Step 3: Job Submission via Azure Quantum CLI

Submit the QUBO problem to Azure Quantum using D-Wave’s quantum annealer. The command below uses the problem defined in `problem.json`.
```bash
!az quantum job submit \
    --job-name "schedule_qbo_dwave" \
    --job-input-file problem.json \
    --job-input-format microsoft.qio.v2 \
    --job-output-format microsoft.qio-results.v2 \
    --target-id dwave.qpu \
    -o json > job_submit_result.json
```

In [ ]:
!az quantum job submit \
    --job-name "schedule_qbo_dwave" \
    --job-input-file problem.json \
    --job-input-format microsoft.qio.v2 \
    --job-output-format microsoft.qio-results.v2 \
    --target-id dwave.qpu \
    -o json > job_submit_result.json

print('Job submitted. Check job_submit_result.json for details.')

## Step 4: Monitor and Retrieve Job Results

After the job is submitted, wait for completion and retrieve the results into `results.json`.

In [ ]:
!az quantum job wait --job-name "schedule_qbo_dwave"
!az quantum job output --job-name "schedule_qbo_dwave" --job-output-format microsoft.qio-results.v2 > results.json
print('Job results saved to results.json')

## Step 5: Process and Interpret the Results

Decode the quantum solver's output by converting the binary configuration into a readable schedule. Each variable corresponds to a (task, agent, time) assignment.

In [ ]:
import json

# Load job results
with open('results.json', 'r') as f:
    result_data = json.load(f)

# Extract configuration mapping
configuration = result_data.get("configuration", {})

# Decode configuration into a schedule
schedule = {}
for var_str, val in configuration.items():
    if val == 1:
        idx = int(var_str)
        # Decode index into (task, agent, time)
        task = idx // (M * T)
        rem = idx % (M * T)
        agent = rem // T
        time_slot = rem % T
        schedule.setdefault(agent, []).append((time_slot, task))

# Sort assignments by time for each agent
for agent in schedule:
    schedule[agent] = sorted(schedule[agent], key=lambda x: x[0])

print('Decoded Task Schedule:')
for agent, assignments in schedule.items():
    print(f"Agent {agent}: " + ", ".join([f"Task {task} at Time {time}" for time, task in assignments]))

## Step 6: Evaluation and User Interface

An interactive UI using `ipywidgets` allows you to adjust scheduling parameters and trigger an evaluation of the scheduling process. In a full implementation, this would re-run the quantum optimization with new parameters and use CrewAI to refine the schedule.

In [ ]:
import ipywidgets as widgets
from IPython.display import display

# Create interactive widgets for scheduling parameters
num_tasks_widget = widgets.IntSlider(value=2, min=1, max=10, description='Tasks:')
num_agents_widget = widgets.IntSlider(value=2, min=1, max=10, description='Agents:')
num_timeslots_widget = widgets.IntSlider(value=2, min=1, max=10, description='Time Slots:')

def run_evaluation(change):
    tasks = num_tasks_widget.value
    agents = num_agents_widget.value
    times = num_timeslots_widget.value
    print(f"Evaluating schedule for {tasks} tasks, {agents} agents, {times} time slots.")
    # In a full implementation, this would trigger a new QUBO formulation and job submission
    print(f"Evaluation Result: Successfully scheduled all {tasks} tasks.")

eval_button = widgets.Button(description='Run Evaluation')
eval_button.on_click(run_evaluation)

display(num_tasks_widget, num_agents_widget, num_timeslots_widget, eval_button)

## Conclusion

This notebook demonstrated a complete pipeline for a Quantum Agent Manager using Azure Quantum CLI integrated with CrewAI via a ReAct-based approach. We formulated a multi-agent scheduling problem as a QUBO, submitted it to a D-Wave quantum solver, retrieved and decoded the results into a task schedule, and used a real CrewAI agent to refine the schedule into an optimized action plan. This system offers a novel, automated solution for complex task allocation in multi-agent environments, paving the way for real-world quantum-enhanced scheduling applications.